<a href="https://colab.research.google.com/github/danmaher067/week6/blob/main/Week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install modin

     |████████████████████████████████| 583kB 5.8MB/s 
     |████████████████████████████████| 9.9MB 33.0MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.3 which is incompatible.
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


In [4]:
pip install 'fsspec>=0.3.3'

     |████████████████████████████████| 112kB 4.5MB/s 


In [2]:
import pandas as pd
import modin.pandas as mpd
import dask.dataframe as dd

In [6]:
%%time
df_pd=pd.read_csv('Final_Cab_Data_Combined.csv')

CPU times: user 209 ms, sys: 68.1 ms, total: 277 ms
Wall time: 280 ms


In [11]:
pip install modin[all]

     |████████████████████████████████| 48.5MB 81kB/s 
     |████████████████████████████████| 17.2MB 258kB/s 
     |████████████████████████████████| 645kB 48.1MB/s 
     |████████████████████████████████| 133kB 55.6MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 1.8MB 45.8MB/s 
     |████████████████████████████████| 204kB 47.5MB/s 
     |████████████████████████████████| 1.3MB 41.2MB/s 
     |████████████████████████████████| 133kB 34.8MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 81kB 6.0MB/s 
     |████████████████████████████████| 3.1MB 36.4MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 4.1MB 38.8MB/s 
     |████████████████████████████████| 122kB 48.2MB/s 
     |████████████████████████████████| 9.5MB 38.4MB/s 
     |████████████████████████████████| 296kB 48.8MB/s 
   

In [3]:

%%time
df_mpd = mpd.read_csv('Final_Cab_Data_Combined.csv')

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



CPU times: user 347 ms, sys: 123 ms, total: 470 ms
Wall time: 5.04 s


In [4]:
%%time
df_dpd=dd.read_csv('Final_Cab_Data_Combined.csv')

CPU times: user 21.8 ms, sys: 1.68 ms, total: 23.5 ms
Wall time: 33.6 ms
